In [1]:
# -*- coding: utf-8 -*-
import os
import re
import csv
import sys
import glob
import errno
import spacy
import random
import sqlite3
import subprocess
import pandas as pd
import en_core_web_sm
import pymysql.cursors
import gensim, logging
from __future__ import division
from operator import itemgetter
from gensim.models import FastText
from gensim.models import Word2Vec
from gensim.models import KeyedVectors
from abbreviations import get_abbreviations

In [2]:
# Avoid ascii error

reload(sys)
sys.setdefaultencoding('utf8')

In [3]:
#read csv file 

def get_candidates(model, file_name):

    keywords = list()
    with open(file_name, 'rb') as csvfile:
        spamreader = csv.reader(csvfile, delimiter='|')
        for row in spamreader:
            for item in row: 
                if '\N' in item: 
                    pass 
                elif len(item) == 0: 
                    pass
                else:
                    if ";" in item: 
                        x = item.split(";")
                        for item in x:
                            line_string = item.strip()
                            keywords.append(line_string)
                    else: 
                        keywords.append(item)
                        
    modified_keywords = list()
    for word in keywords: 
        if word in model.wv.vocab:
            modified_keywords.append(word)
        else:
            pass
        
    return modified_keywords


In [4]:
# keywords = polymers # These already give better numbers than PS and polystyrene after looking at clusters

In [5]:
nlp = spacy.load("en")

#Adding more words to vocab in order to be ignored
# nlp.vocab["diblock"].is_stop = False
# nlp.vocab["g/mol"].is_stop = False
# nlp.vocab["kg/mol"].is_stop = False
# nlp.vocab["copolymers"].is_stop = False
# nlp.vocab["copolymer"].is_stop = False
# nlp.vocab["vpol"].is_stop = False

In [6]:
def remove_common_pos(word):
    '''Checks if a word ends with a common noun, verb, adverb form'''
    ending_with = ["ity", "ize", "ized", "tion", "ly", "sion", "ure", "graph", "ing", "cal", "ent", "ed", "nt", "ic", "ies", "tions", "ar", "ous", "al", "able", "nce", "try", "one", "ive", "or", "sis", "ter", "ory", "mer", "imides"]
    for end in ending_with:
        if word.endswith(end):
            return True
            
    return False

def is_ascii(s):
    '''Checks if input is english (filtering out Greek symbols)'''
    return all(ord(c) < 128 for c in s)

def is_number(n):
    try:
        float(n)
        return True
    except ValueError:
        return False

In [7]:
def clean_up_candidates(candidates, n, ignore_case):
    '''Cleaning up candidates that are not polymer names but appear in the same context'''
    
    #Identify context words that could be removed
    context_file = "./evaluation/misc/context_words.txt"
    context_words = [line.strip() for line in open(context_file).readlines()]
    
    #Remove unwanted punctuations
    candidates = [(k.strip(',.:-;'),v) for (k,v) in candidates]
    
    #Remove () from candidates enclosed in parenthesis
    for i in range(len(candidates)):
        cand = candidates[i][0]
        
        if cand[0] == '(' and cand[-1] ==')':
            updated_candidate = (cand[1:-1], candidates[i][1])
            candidates[i] = updated_candidate
        
    #check if case-sensitivity is required
    if ignore_case is True:
        candidates = [(str.lower(k),k,v) for (k, v) in candidates]
        
    #Remove candidate that appear in regular English dictionary  
    #But don't remove common polymers 
    
    common_polys = ['polyethylene', 'polyurethane', 'polypropylene', 'polyester', 'PS', 'polystyrene', 'PLA', 'PI', 'PET', 'PVP', 'PEG', 'cellulose', 'PAN', 'methyl'] #These are polymers that could appear within spacy vocab
    common_polys = [str.lower(polymer) for polymer in common_polys]
    
    junk_vals = context_words
    for (candidate_lower, candidate, sim) in candidates:
        if not is_ascii(candidate_lower):
            junk_vals.append(candidate_lower)
            
        elif (candidate in nlp.vocab) and str.lower(candidate) not in common_polys:
            junk_vals.append(candidate_lower)
            
            
        else:
            junk = False
            items = re.split(' |:|;|/',candidate_lower)
            for item in items:
                    
                if item != "poly" and is_number(item) is False and ("standard" in item or (item in nlp.vocab and item not in common_polys)):
                    junk = True
                    break
                    
            if junk is True:
                junk_vals.append(candidate_lower)
        
    
    candidates = [(candidate_lower, candidate, sim) for (candidate_lower,candidate, sim) in candidates if candidate_lower not in junk_vals]
#     candidates.sort(key=operator.itemgetter(2))
    candidates = sorted(candidates, key=lambda x : x[2])
    dict_cand = {}
    dict_cand_lower = {}
    
    for cand in candidates:
        if cand[0] in dict_cand_lower.keys():
            cand_with_case = dict_cand_lower[cand[0]]
            dict_cand[cand_with_case] = max(dict_cand[cand_with_case], cand[2])
        
        else:
            dict_cand[cand[1]] = cand[2]
            dict_cand_lower[cand[0]] = cand[1] #Mapping lower case to acual polymer
    # sort the polymers by similarity similar to the keywords 

    candidates = sorted(list(dict_cand.items()), key=itemgetter(1), reverse=True) 
    
    # take the top n candidates and generate a list
    candidate_list = [cand for (cand,v) in candidates[:n]]
    
    candidate_list = list(set(candidate_list)) # not really required
    return candidate_list

In [8]:
def get_candidates_similar(keyword_list, model, n, ignore_case=True):
    ''' Gets candidates that are similar to a list of keywords'''
    candidates = []
    for keyword in keyword_list:
        candidates += model.wv.most_similar(positive=keyword, topn=1500)

    candidate_list = clean_up_candidates(candidates, n, ignore_case)
    return candidate_list

In [9]:
def write_candidates_to_file(model, input_file_name, file_name, n=800, ignore_case=True):
    ''' Gets the list of candidates from get_candidates_similar and writes to file '''
    
    modified_keywords = get_candidates(model, input_file_name)
    cand_list = get_candidates_similar(modified_keywords, model, n, ignore_case)
    file = open(file_name, 'w+')
    
    for cand in cand_list:
        file.write(cand + '\n')

In [10]:
def train_FT_model(in_file, sg=1, word_ngrams=5, iter=10, window=5):
    ''' Train a FastText skipgram model for word embedding using subword information'''
    
    f = open(in_file, 'r')
    lines = f.readlines()

    sentence_token_list = []
    for line in lines:
        tokens = re.split(r"\s+(?=[^()]*(?:\(|$))", line.rstrip("."))
        tokens = tokens[1:] # First is docid 
        sentence_token_list.append([token.strip(',.:-; ') for token in tokens])
    print (sentence_token_list)  
    model = FastText(sentence_token_list, size=120, window=window, min_count=1, workers=4,sg=sg, word_ngrams=word_ngrams, iter=iter, seed=23)
    return model

In [11]:
# #Comment this cell after saving for the first time

# model_dir = "./models"
# if not os.path.exists(model_dir):
#     os.makedirs(model_dir)
    
# model = train_FT_model("./data/FT_WE_data_input.txt", sg=0, word_ngrams=5)#sg = 0 => cbow model
# model.save('./models/FT_cbow.bin')

In [12]:
model = FastText.load('./models/FT_cbow.bin')

In [13]:
write_candidates_to_file(model, "scraped_polymer_names.txt", "./evaluation/candidates/listformat/second_FT_candidates_500.txt", n=500)